In [1]:
import pandas as pd

df = pd.read_csv('data/train_data.csv')
df = df.dropna()

df.head(5)

,x,y,z,t0,t1,t2,t3,xc_px,yc_px,xc,yc,diagonal
7,0.16,-0.19,0.12,30.100908,33.948155,75.041009,41.092854,-275.0,145.0,4.950,2.610,73.389373
8,0.17,-0.18,0.12,33.363423,34.952635,76.655631,41.702995,-264.0,160.0,4.752,2.880,72.801099
9,0.18,-0.17,0.12,36.636577,34.952635,76.655631,41.702995,-249.0,178.0,4.482,3.204,72.780492
10,0.19,-0.16,0.12,39.899092,33.948155,75.041009,41.092854,-235.0,191.0,4.230,3.438,72.835431
11,0.20,-0.15,0.12,43.130102,31.753588,71.469230,39.715641,-222.0,208.0,3.996,3.744,75.239617


In [18]:
from utils.functions import mapping, ikine
l1, l2, l3 = 0.1, 0.124, 0.06 
import numpy as np

def simulate_travel(start:tuple[float, float], end:tuple[float, float], travel_points:list[float]):
    xi, yi = start
    xf, yf = end
    
    t0, t1, t2, t3, _, _ = mapping(ikine([xi, yi, .12], l1, l2, l3))
    data = {
        "t0i":  t0,
        "t1i":  t1,
        "t2i":  t2,
        "t3i":  t3,
        "xi"  : xi,
        "yi"  : yi,
    }
    
    
    for point in travel_points:
        x = xi + point * (xf-xi)
        y = yi + point * (yf-yi)
        
        t0, t1, t2, t3, _, _ = mapping(ikine([x, y, .12], l1, l2, l3))
        data.update({
            f"t0{point:.2f}" : t0,
            f"t1{point:.2f}":  t1,
            f"t2{point:.2f}":  t2,
            f"t3{point:.2f}":  t3,
            f"x{point:.2f}"  : x,
            f"y{point:.2f}"  : y,
        })
    
    t0, t1, t2, t3, _, _ = mapping(ikine([x, y, .12], l1, l2, l3))
    data.update({
        f"t0f": t0,
        f"t1f": t1,
        f"t2f": t2,
        f"t3f": t3,
        f"xf"  : xf,
        f"yf"  : yf,
    })
    
    return data

def get_data_train(n_samples=None, travel_points=[0.3, 0.5, 0.75, 0.825, 0.9, 0.95], x_start:float=0.035, y_start:float=0)->pd.DataFrame:
    if n_samples is None:
        points = df[['x', 'y']].values
    else:
        x_range, y_range = (df['x'].min(), df['x'].max()), (df['y'].min(), df['y'].max())
        points = np.column_stack((np.random.uniform(*x_range,  n_samples), np.random.uniform(*y_range, n_samples)))
        
    data = []
    for point in points:
        try:
            travel = simulate_travel((x_start, y_start), point, travel_points)
            data.append(travel)
        except:
            pass
    
    return pd.DataFrame(data)

In [ ]:
narx_train_dataset = get_data_train() # RETORNA todos os x e Y do dataset de treinamento
narx_train_values = get_data_train(10_000)


      t0i         t1i        t2i       t3i     xi  yi      t00.30      t10.30  \
0    80.0  163.649548  171.03985  7.390302  0.035   0   41.825326  133.444339   
1    80.0  163.649548  171.03985  7.390302  0.035   0   44.426445  133.082299   
2    80.0  163.649548  171.03985  7.390302  0.035   0   46.988962  132.605931   
3    80.0  163.649548  171.03985  7.390302  0.035   0   49.503742  132.017756   
4    80.0  163.649548  171.03985  7.390302  0.035   0   51.962718  131.320826   
..    ...         ...        ...       ...    ...  ..         ...         ...   
699  80.0  163.649548  171.03985  7.390302  0.035   0   95.767183  159.702987   
700  80.0  163.649548  171.03985  7.390302  0.035   0   96.898649  160.991086   
701  80.0  163.649548  171.03985  7.390302  0.035   0  100.794115  160.562048   
702  80.0  163.649548  171.03985  7.390302  0.035   0   92.835609  161.328372   
703  80.0  163.649548  171.03985  7.390302  0.035   0   91.956584  160.028731   

         t20.30     t30.30 